Mount Google Drive if running in Colab.

In [ ]:
try:
    from google.colab import drive

    drive.mount("/content/drive")
    %pip install cvlib
except ModuleNotFoundError:
    pass

Import libraries.

In [ ]:
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import cv2
import os
import cvlib as cv

Download Arun Ponnusamy's model.

In [ ]:
dwnld_link = "https://github.com/arunponnusamy/cvlib/releases/download/v0.2.0/gender_detection.model"
model_path = get_file(
    "gender_detection.model",
    dwnld_link,
    cache_subdir="pre-trained",
    cache_dir=os.getcwd(),
)

Define our folder for the screenshots.

In [ ]:
images_folder = "./drive/MyDrive/Screenshots/"

In [ ]:
def load_film_images(path):
    """Loads all the images in a folder.

    Args:
        path (str): The path to the image folder.

    Returns:
        list: A list of the images in the folder loaded.
    """
    images = []
    for image_filename in os.listdir(path):
        image = cv2.imread(os.path.join(path, image_filename))
        if image is None:
            print("Failed to load as image:", image_filename)
            continue
        images.append(image)
    return images

Go through the screenshots folders and create a list of the folder names.

In [ ]:
movie_folder_names = []
for movie_folder in os.listdir(images_folder):
    if movie_folder == "output":
        # Ignore the output folder to prevent it processing images more than once
        continue
    if os.path.isdir(os.path.join(images_folder, movie_folder)):
        # Add the folder name to the list only if it is a folder
        movie_folder_names.append(movie_folder)

List the folders found in the Screenshots folder.

In [ ]:
print(movie_folder_names)

Load the model.

In [ ]:
# load pre-trained model
model = load_model(model_path)

Create an output folder if one doesn't already exist.

In [ ]:
output_dir = "./drive/MyDrive/Screenshots/output"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Create a list of categories for our data.

In [ ]:
MEN = "men"
WOMEN = "women"
MIXED = "mixed"
categories = [MEN, WOMEN, MIXED]

Create folders within our output folders for the different films. And then create folders within those folders for men/women/mixed.

In [ ]:
for movie_name in movie_folder_names:
    movie_output_path = os.path.join(output_dir, movie_name)
    if not os.path.exists(movie_output_path):
        os.mkdir(movie_output_path)
    for category in categories:
        category_path = os.path.join(movie_output_path, category)
        if not os.path.exists(category_path):
            os.mkdir(category_path)

Define our classes.

In [ ]:
MAN = "man"
WOMAN = "woman"
classes = [MAN, WOMAN]

Create a helper method to see if the faces found in an image are all male or all female.

In [ ]:
def faces_are_all(gender, images):
    """Checks if the faces are all one gender.

    Args:
        gender (str): The gender that the list of faces should match.
        images (list): A list of the genders of the faces found in an image.

    Returns:
        bool: True if the faces all match the given gender, False otherwise.
    """
    return all([image_gender == gender for image_gender in images])

Choose our file type for the output.

In [ ]:
EXTENSION = ".jpg"

Load the images and classify them.

In [ ]:
for movie_name in movie_folder_names:
    images = load_film_images(os.path.join(images_folder, movie_name))
    counter = 0
    for image in images:
        # detect faces in the image
        face, confidence = cv.detect_face(image, enable_gpu=True)

        if len(face) == 0:
            # try again with next image if no faces were found
            continue

        # create a list for the gender of the people found in the image
        genders = []

        # loop through detected faces
        for idx, f in enumerate(face):
            try:
                # get corner points of face rectangle
                (startX, startY) = f[0], f[1]
                (endX, endY) = f[2], f[3]

                # draw rectangle over face
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

                # crop the detected face region
                face_crop = np.copy(image[startY:endY, startX:endX])

                # preprocessing for gender detection model
                face_crop = cv2.resize(face_crop, (96, 96))
                face_crop = face_crop.astype("float") / 255.0
                face_crop = img_to_array(face_crop)
                face_crop = np.expand_dims(face_crop, axis=0)

                # apply gender detection on face
                conf = model.predict(face_crop)[0]

                # get label with max accuracy
                idx = np.argmax(conf)
                label = classes[idx]

                # add current gender to the list of genders in the image
                genders.append(classes[idx])

                label = "{}: {:.2f}%".format(label, conf[idx] * 100)

                Y = startY - 10 if startY - 10 > 10 else startY + 10

                # write label and confidence above face rectangle
                cv2.putText(
                    image,
                    label,
                    (startX, Y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2,
                )
            except Exception as e:
                print(e)
                print(f)
                continue

        if faces_are_all(MAN, genders):
            # save in mem folder when faces that have been detected are all male
            cv2.imwrite(
                os.path.join(output_dir, movie_name, MEN, str(counter) + ".jpg"), image
            )
        elif faces_are_all(WOMAN, genders):
            # save in women folder when faces that have been detected are all female
            cv2.imwrite(
                os.path.join(output_dir, movie_name, WOMEN, str(counter) + ".jpg"),
                image,
            )
        else:
            # if faces aren't all male or all female then they must be mixed, so images is saved to mixed folder
            cv2.imwrite(
                os.path.join(output_dir, movie_name, MIXED, str(counter) + ".jpg"),
                image,
            )
        # increase counter to prevent overwritting previous image
        counter += 1
        print("saved image")
    # free up space at the end
    images.clear()

The classified images should now be in a folder called output in the Screenshots folder.